# data

> Fill in a module description here


In [ ]:
#| default_exp vision.downloader

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import os
import random
import torch
import torchvision.transforms as transforms
from fedai.data.downloader import *
from fedai.utils import *
from fastcore.utils import patch

In [ ]:
#| export
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#| export
class VisionDownloader(BaseDownloader):  # noqa: F405
    def __init__(self, cfg, transform=None):
        self.transform_norm_mapping = {
            "CIFAR10": ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            "CIFAR100": ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            "MNIST": ((0.5,), (0.5,)),
            "FashionMNIST": ((0.5,), (0.5,)),
            "EMNIST": ((0.5,), (0.5,)),
        }
        if not transform:
            self.transform = transforms.Compose(
                [transforms.ToTensor(), 
                transforms.Normalize(self.transform_norm_mapping[cfg.data.name][0],
                                    self.transform_norm_mapping[cfg.data.name][1])])

        else:
            self.transform = transform
        super().__init__(cfg)

    

In [ ]:
#| export
@patch
def load_data(self: VisionDownloader):
    if not os.path.exists(self.cfg.data.dir_path):
        os.makedirs(self.cfg.data.dir_path)
    
    ds_class = get_class("torchvision.datasets", self.cfg.data.name)  # noqa: F405
    # Setup directory for train/test data
    if self.check():
        return
    
    trainset = ds_class(
        root=os.path.join(self.cfg.data.dir_path, self.cfg.data.name, "rawdata"), train=True, download=True, transform=self.transform)
    testset = ds_class(
        root=os.path.join(self.cfg.data.dir_path, self.cfg.data.name, "rawdata"), train=False, download=True, transform=self.transform)
    
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=len(trainset.data), shuffle=False)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=len(testset.data), shuffle=False)

    for _, train_data in enumerate(trainloader, 0):
        trainset.data, trainset.targets = train_data
    for _, test_data in enumerate(testloader, 0):
        testset.data, testset.targets = test_data

    dataset_image = []
    dataset_label = []

    dataset_image.extend(trainset.data.cpu().detach().numpy())
    dataset_image.extend(testset.data.cpu().detach().numpy())
    dataset_label.extend(trainset.targets.cpu().detach().numpy())
    dataset_label.extend(testset.targets.cpu().detach().numpy())
    dataset_image = np.array(dataset_image)
    dataset_label = np.array(dataset_label)

    num_classes = len(set(dataset_label))
    print(f'Number of classes: {num_classes}')

    return dataset_image, dataset_label

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()